<a href="https://colab.research.google.com/github/DUNGTK2004/Pytorch-Tutorial/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Convert MNIST Image Files into a Tensor of 4-dimensions (# of images, Height, Width, Color)
transform = transforms.ToTensor()

In [3]:
# Train Data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:06<00:00, 1482270.12it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 134986.38it/s]


Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1277973.46it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4025893.65it/s]

Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [4]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
pwd

'/content'

In [8]:
ls

sample_data/


In [9]:
cd ../

/


In [10]:
pwd

'/'

In [11]:
ls

bin@       cuda-keyring_1.0-1_all.deb  home/    lib64@   NGC-DL-CONTAINER-LICENSE  run/   tmp/
boot/      datalab/                    kaggle/  libx32@  opt/                      sbin@  tools/
cnn_data/  dev/                        lib@     media/   proc/                     srv/   usr/
content/   etc/                        lib32@   mnt/     root/                     sys/   var/


In [12]:
cd cnn_data

/cnn_data


In [13]:
pwd

'/cnn_data'

In [14]:
ls

MNIST/


In [15]:
cd ../

/


In [16]:
ls

bin@       cuda-keyring_1.0-1_all.deb  home/    lib64@   NGC-DL-CONTAINER-LICENSE  run/   tmp/
boot/      datalab/                    kaggle/  libx32@  opt/                      sbin@  tools/
cnn_data/  dev/                        lib@     media/   proc/                     srv/   usr/
content/   etc/                        lib32@   mnt/     root/                     sys/   var/


In [17]:
cd content/

/content


In [18]:
pwd

'/content'

In [19]:
# Create a small batch size for images...let's say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)

In [20]:
# Define Our CNN Model
# Describe convolutional layer and what it's doing (2 convolutional layers)
# This is just an example in the next video we'll build out the actual model
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [21]:
# Grab 1 MNIST record/image
for i, (X_Train, y_train) in enumerate(train_data):
  break

In [22]:
X_Train.shape

torch.Size([1, 28, 28])

In [23]:
x = X_Train.view(1, 1, 28, 28)

In [24]:
# Perform our first convolution
x = F.relu(conv1(x)) # Rectified Linear Unit for our activation function

In [25]:
# 1 single image, 6 is filters we asked for , 26x26
x.shape

torch.Size([1, 6, 26, 26])

In [26]:
# pass thru the pooling layer
x = F.max_pool2d(x, 2, 2) # kernal of 2 and stride of 2

In [27]:
x.shape # 26 / 2 = 13

torch.Size([1, 6, 13, 13])

In [28]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [29]:
x.shape # Again, we didn't set padding so we lose 2 pixles around the outside of the image

torch.Size([1, 16, 11, 11])

In [30]:
# Pooling layer
x = F.max_pool2d(x, 2, 2)

In [31]:
# 11 / 2 = 5.5 but we have to round down, because you can't invent data to round up
x.shape

torch.Size([1, 16, 5, 5])

In [32]:
# Model Class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)
    # Fully Connected Layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2) # 2x2 kernal and stride 2
    # Second Pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2) # 2x2 kernal and stride 2

    # Re-View to flatten it out
    X = X.view(-1, 16*5*5) # negative one so that we can vary the batch size

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)

In [33]:
# Create an Instance of our Model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [34]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Smaller the Learning Rate, longer its gonna take to train.

In [38]:
import time
start_time = time.time()

# Create Variables To Tracks Things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For loop of Epochs
for i in range(epochs):
    trn_corr = 0
    tst_corr = 0

    # Train
    for b, (X_train, y_train) in enumerate(train_loader):
      b += 1 # start our batches at 1
      y_pred = model(X_train) # get predicted values from the training set. Not flattened 2D
      loss = criterion(y_pred, y_train) # how off are we? Compare the predictions to correct answers in y_train

      predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions. Indexed off the first point
      batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True = 1, False = 0, sum those up
      trn_corr += batch_corr # keep track as we go along in training.

      # Update our parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Print out some results
      if b%600 == 0:
        print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

    train_losses.append(loss)
    train_correct.append(trn_corr)

    # Test
    with torch.no_grad(): # No gradient so we don't update our weights and biases with test
      for b, (X_test, y_test) in enumerate(test_loader):
        y_val = model(X_test)
        predicted = torch.max(y_val.data, 1)[1] # Adding up correct predictions
        tst_corr += (predicted == y_test).sum() # T=1 F=0 and sum away

    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)

current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes!')

Epoch: 0  Batch: 600  Loss: 0.0052002109587192535
Epoch: 0  Batch: 1200  Loss: 0.030193308368325233
Epoch: 0  Batch: 1800  Loss: 0.009836439043283463
Epoch: 0  Batch: 2400  Loss: 0.08481411635875702
Epoch: 0  Batch: 3000  Loss: 0.0014447346329689026
Epoch: 0  Batch: 3600  Loss: 0.003043821547180414
Epoch: 0  Batch: 4200  Loss: 0.3956584930419922
Epoch: 0  Batch: 4800  Loss: 0.05811353772878647
Epoch: 0  Batch: 5400  Loss: 0.008319977670907974
Epoch: 0  Batch: 6000  Loss: 0.00669638067483902
Epoch: 1  Batch: 600  Loss: 0.12239543348550797
Epoch: 1  Batch: 1200  Loss: 0.003355450229719281
Epoch: 1  Batch: 1800  Loss: 0.0010270519414916635
Epoch: 1  Batch: 2400  Loss: 0.0417042002081871
Epoch: 1  Batch: 3000  Loss: 0.004491274245083332
Epoch: 1  Batch: 3600  Loss: 0.0022446836810559034
Epoch: 1  Batch: 4200  Loss: 0.0019128683488816023
Epoch: 1  Batch: 4800  Loss: 0.004196741618216038
Epoch: 1  Batch: 5400  Loss: 0.04689784720540047
Epoch: 1  Batch: 6000  Loss: 0.28048065304756165
Epoch: 